In [1]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
# Weights
w_c, w_l, w_s = sp.symbols('w_c w_l w_s', positive=True)
# Token balances
x_c, x_l, x_s = sp.symbols('x_c x_l x_s', positive=True)
# Prices - in units of locked up collateral
v = sp.symbols('v', positive=True) 
# Swap fee in fraction of input tokens
s_f = sp.symbols('s_f')
# Decimal scaling 
d_c, d_p = sp.symbols('d_c d_p', positive=True)
# Collateral backing L + S 
# e.g. for $100.000_000_000_000_000_000 = 1.000_000 mt
# this is 100*10**18 / 1*10**6 = 10**14
C = sp.symbols('C', positive=True)

In [4]:
def calc_spot_price(bI, wI, bO, wO, sF=0):
    """
    calcSpotPrice                                                                             //
     sP = spotPrice                                                                            //
     bI = tokenBalanceIn                ( bI / wI )         1                                  //
     bO = tokenBalanceOut         sP =  -----------  *  ----------                             //
     wI = tokenWeightIn                 ( bO / wO )     ( 1 - sF )                             //
     wO = tokenWeightOut                                                                       //
     sF = swapFee 
    """
    return (bI/wI)/(bO/wO)/(1-sF)

In [5]:
# e.g. condition for spot price of long token without fees to be v*C
calc_spot_price(x_c, w_c, x_l, w_l) - v*C

-C*v + w_l*x_c/(w_c*x_l)

In [6]:
sol = sp.solve(
    [w_c + w_l + w_s - 1,   # Weights sum to 1
     calc_spot_price(x_c, w_c, x_l, w_l, 0) - v*C,      # Long token price = v*C
     calc_spot_price(x_c, w_c, x_s, w_s, 0) - (1-v)*C,  # Short token price = (1-v)*C
    ]
     , [w_c, w_l, w_s])

In [7]:
sol[w_c]

x_c/(C*v*x_l - C*x_s*(v - 1) + x_c)

In [8]:
sol[w_l]

C*v*x_l/(C*v*x_l - C*x_s*(v - 1) + x_c)

In [9]:
sol[w_s]

-C*x_s*(v - 1)/(C*v*x_l - C*x_s*(v - 1) + x_c)

In [10]:
# Full set of weights needed to set the spot prices of long and short tokens
sol

{w_c: x_c/(C*v*x_l - C*x_s*(v - 1) + x_c),
 w_l: C*v*x_l/(C*v*x_l - C*x_s*(v - 1) + x_c),
 w_s: -C*x_s*(v - 1)/(C*v*x_l - C*x_s*(v - 1) + x_c)}

In [11]:
# Check spot prices
calc_spot_price(x_c, w_c, x_l, w_l, s_f).subs(sol) # Long spot price

C*v/(1 - s_f)

In [12]:
calc_spot_price(x_c, w_c, x_s, w_s, s_f).subs(sol)

-C*(v - 1)/(1 - s_f)

In [13]:
def calc_out_given_in(bO, wO, bI, wI, aI, sF=0):
    """
    calcOutGivenIn                                                                            //
     aO = tokenAmountOut                                                                       //
     bO = tokenBalanceOut                                                                      //
     bI = tokenBalanceIn              /      /            bI             \    (wI / wO) \      //
     aI = tokenAmountIn    aO = bO * |  1 - | --------------------------  | ^            |     //
     wI = tokenWeightIn               \      \ ( bI + ( aI * ( 1 - sF )) /              /      //
     wO = tokenWeightOut                                                                       //
     sF = swapFee 
    """
    return bO*(1-(bI/(bI + (aI*(1-sF))))**(wI/wO))

In [14]:
def calc_in_given_out(bO, wO, bI, wI, aO, sF=0):
    """
    calcInGivenOut                                                                            //
     aI = tokenAmountIn                                                                        //
     bO = tokenBalanceOut               /  /     bO      \    (wO / wI)      \                 //
     bI = tokenBalanceIn          bI * |  | ------------  | ^            - 1  |                //
     aO = tokenAmountOut    aI =        \  \ ( bO - aO ) /                   /                 //
     wI = tokenWeightIn           --------------------------------------------                 //
     wO = tokenWeightOut                          ( 1 - sF )                                   //
     sF = swapFee                                               
    """
    return bI*((bO/(bO - aO))**(wO/wI) - 1)/(1-sF)

In [15]:
# Input token amounts for swaps
a_c, a_l, a_s = sp.symbols('a_c a_l a_s', positive=True)

In [16]:
n, d = sp.fraction(w_c.subs(sol))

In [17]:
w_c.subs(sol).subs({d: 1})

x_c

# Check for arbitrage: buy L, buy S, redeem for C


In [18]:
# Buy long tokens for a_c collateral tokens
x_l_O = calc_out_given_in(x_l, w_l, x_c, w_c, a_c, s_f).subs(sol).simplify()

In [19]:
# Updated balance in pool
x_l_1 = x_l - x_l_O
x_c_1 = x_c + a_c

In [20]:
# Buy same amount of short tokens 
x_c_I = calc_in_given_out(x_s, w_s, x_c_1, w_c, x_l_O, s_f).subs(sol).simplify()

In [21]:
trade_cost = (a_c + x_c_I).simplify() # Buy long from pool, buy same amount of short
trade_revenue = (C*x_l_O).simplify()  # Redeem long-short pair for underlying collateral

In [22]:
# Input as fraction of balance i.e. a_c = f_c * x_c
f_c, f_l, f_s = sp.symbols('f_c f_l f_s', positive=True)

In [23]:
trade_cost.subs({a_c: f_c*x_c}).simplify()

f_c*x_c + (1 - (x_s/(x_l*(x_c/(-f_c*s_f*x_c + f_c*x_c + x_c))**(x_c/(C*v*x_l)) - x_l + x_s))**(-C*x_s*(v - 1)/x_c))*(f_c*x_c + x_c)/(s_f - 1)

In [24]:
# Look at first and second order revenue - cost 
sp.series((trade_revenue - trade_cost).subs({a_c: f_c*x_c}).simplify(), f_c, n=2).coeff(f_c, 1).simplify()

-s_f*x_c/v

In [39]:
n=2
sp.series((trade_revenue - trade_cost).subs({a_c: f_c*x_c}).simplify(), f_c, n=n+1).coeff(f_c, n).simplify().collect(v)

x_c*(2*C*s_f*v**2*x_l*x_s + C*s_f*x_l*x_s - C*x_l*x_s - s_f**2*x_c*x_s + s_f*x_c*x_l + s_f*x_c*x_s + v*(-C*s_f**2*x_l*x_s - C*s_f*x_l*x_s - s_f*x_c*x_l + s_f*x_c*x_s + x_c*x_l - x_c*x_s) - x_c*x_l)/(2*C*v**2*x_l*x_s)

# Check for arbitrage: mint C, sell L, sell S 

In [43]:
# Buy coin tokens for long tokens
x_c_O = calc_out_given_in(x_c, w_c, x_l, w_l, a_l, s_f).subs(sol).simplify()

In [44]:
# Updated balance in pool
y_l_1 = x_l + a_l
y_c_1 = x_c - x_c_O

In [47]:
# Buy coin tokens for same amount of short tokens 
x_c_O_2 = calc_out_given_in(y_c_1, w_c, x_s, w_s, a_l, s_f).subs(sol).simplify()

In [48]:
trade_cost_mint = (a_l * C).simplify() # Mint long and short 
trade_revenue_mint = (x_c_O + x_c_O_2).simplify()  # Redeem long-short pair for underlying collateral

In [51]:
trade_cost_mint

C*a_l

In [52]:
trade_revenue_mint

-x_c*(x_l/(-a_l*s_f + a_l + x_l))**(C*v*x_l/x_c)*(x_s/(-a_l*s_f + a_l + x_s))**(C*x_s/x_c)*(x_s/(-a_l*s_f + a_l + x_s))**(-C*v*x_s/x_c) + x_c

In [55]:
sp.series((trade_revenue_mint - trade_cost_mint).subs({a_l: f_l*x_l}).simplify(), f_l, n=3)#.coeff(f_l, 1).simplify()

f_l**2*(-C**2*s_f**2*x_l**2/(2*x_c) + C**2*s_f*x_l**2/x_c - C**2*x_l**2/(2*x_c) + C*s_f**2*v*x_l**2/(2*x_s) - C*s_f**2*v*x_l/2 - C*s_f**2*x_l**2/(2*x_s) - C*s_f*v*x_l**2/x_s + C*s_f*v*x_l + C*s_f*x_l**2/x_s + C*v*x_l**2/(2*x_s) - C*v*x_l/2 - C*x_l**2/(2*x_s)) - C*f_l*s_f*x_l + O(f_l**3)

Consider now pair balance and difference with $x_l = (x_p + x_d)/2$ and $x_s = (x_p - x_d)/2$ so that we can introduce a budget constraint $x_c + C*x_p = b_c$

In [26]:
x_p, x_d = sp.symbols('x_p x_c')
subs_dict = {x_l: (x_p + x_d)/2, x_s: (x_p - x_d)/2}

In [27]:
b_c = sp.symbols('b_c')
budget_subs_dict = {x_c: b_c - C*x_p}

In [28]:
sol[w_c].subs(subs_dict).simplify()

2*x_c/(2*C*v*x_c - C*x_c + C*x_p + 2*x_c)

In [29]:
sol[w_l].subs(subs_dict).simplify()

C*v*(x_c + x_p)/(2*C*v*x_c - C*x_c + C*x_p + 2*x_c)

In [30]:
sol[w_s].subs(subs_dict).simplify()

C*(v - 1)*(x_c - x_p)/(C*v*(x_c + x_p) + C*(v - 1)*(x_c - x_p) + 2*x_c)

In [31]:
(sol[w_c] + sol[w_l] + sol[w_s]).simplify()

1

In [32]:
sol[w_c].subs(subs_dict).subs(budget_subs_dict).simplify()

(-2*C*x_p + 2*b_c)/(2*C*v*x_c - C*x_c - C*x_p + 2*b_c)

In [33]:
sol[w_c].subs(subs_dict).subs(budget_subs_dict).simplify().diff(x_p).simplify()

2*C*(-2*C*v*x_c + C*x_c - b_c)/(2*C*v*x_c - C*x_c - C*x_p + 2*b_c)**2

In [35]:
?sp.symbols